# 一.简述
> * 使用.from_pretrained导入预训练的config，tokenizer，model,
> * 可以使用AutoModel，AutoTokenier，AutoConfig自动载入预训练内容，但建议还是使用确切的模型类，因为可能会有bug
> * transformer不是用于构建网络结构的工具，如果想自己重构一个模型，还是要使用pytorch之类的
> * single model file 单一模型设计理念,模型前向传播所需的所有代码都在一个且只有一个文件中——称为模型文件

# 二.准备数据
> * 数据集的构建，构建的dataset应该是原始文本的，而dataset的处理可以放到后面模型训练用到的时候做
> * 这边构建的dataset应该是不变的，可以变的是train和validation，每次都应该可以更新到dataset_dict中，而且应该可以使用cache进行判别是否可以更新
> * 具体看datasets_learning.ipynb

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7,8'
os.environ["WANDB_MODE"] = "offline"

In [2]:
from datasets import load_dataset
dataset = load_dataset("glue","cola",split="validation")

/home/zyl/disk/poetry_env/algorithms-ai-IgVB5U1f-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/zyl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


# 三.构建一个Trainer
> * 主要使用Trainer类,传入相关参数
>> * model: 模型结构（PreTrainedModel或者torch.nn.Module），包含forward前向传播，模型类初始化，损失的计算。如果没有提供这个参数，model_init必须传入
>> * args：模型训练的参数，TrainingArguments
>> * data_collator：DataCollator把dataset处理成batch-dataset，会使用default_data_collator，比如DataCollatorWithPadding动态填充
>> * train_dataset：训练集，可以是torch.utils.data.Dataset和torch.utils.data.IterableDataset，可以流处理，但是也要在其他地方进行一些配置，用不到的列会被模型-forward自动删除
>> * eval_dataset：评估集，可以是torch.utils.data.Dataset或者Dict（torch.utils.data.Dataset），也就是如果使用多个dataset，会在评估的时候前面加上前缀
>> * tokenizer：分词PreTrainedTokenizerBase，数据预处理的时候用到，实际包括规范化+预分词+分词，如果提供会自动pad，而且这个独立于模型，方便训练完成后，整体导入模型的时候可以同时导入tokenizer，方便使用
>> * model_init：可调用获得模型的函数，Callable[[], PreTrainedModel]，主要用在超参搜索传入不同参数,因为模型初始化会覆盖model
>> * compute_metrics：可调用的计算指标的函数，Callable[[EvalPrediction], Dict]输入的是EvalPrediction，输出的是指标字典，自定义指标用到，默认trainer不会在训练时自动评估模型的性能
>> * callbacks：可以配置训练的回调函数，List of [`TrainerCallback`],用于自定义训练循环的回调列表，可以在训练过程中自己添加一些操作，也可以删除一些回调，Trainer.remove_callback
>> * optimizers：优化器和学习率调度器，Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR]默认AdamW和get_linear_schedule_with_warmup
>> * preprocess_logits_for_metrics：在评估前处理logits的函数 Callable[[torch.Tensor, torch.Tensor], torch.Tensor]必须传入两个张量logits，labels（默认NONE），所以这个参数在为了配合metrics的时候使用，而且是张量处理，而metrics接收的是array
> * Trainer类的相关属性
>> * model:核心模型
>> * model_wrapped：包裹的模型，比如ddp-model
>> * is_model_parallel：是否模型并行（注意：非数据并行，模型的层放在不同gpu上）
>> * place_model_on_device:默认Flase（模型并行或deepspeed），是否把模型自动放到device上
>> * is_in_train:是否当前的模型处于训练模式，在评估时可能用到这个判断
> * 流程：
>> * 第一步必须构建模型的结构model（传入一些模型参数，比如层数，只有前向传播），然后需要传入一些模型的训练的超参数args（lr等），这样模型以及基本的训练框架就构建好了
>> * 第二步需要传入数据，首先模型的训练集如果较大的时候，可以配置流处理，模型的评估集如果分多个评估集需要弄成一个dict，因为数据集不是batch样的，所以需要data_collator把数据变成batch样的，然后因为数据原始是文本输入，所以要把它变成模型可以接收的input_ids这种形式，用到tokenizer-process，默认使用模型的tokenizer
>> * 第三步可以简单自定义一些训练的操作，比如超参搜索model_init，自定义指标compute_metrics，自定义训练callback，自定义优化器和调度器optimizers，自定义logits的处理preprocess_logits_for_metrics


## 1. 构建模型model
> * 这里的模型指带参数的checkpoint-model，而不是仅仅模型架构（比如bert结构和bert-base-uncased-checkpoint）
> * 模型初始化不仅要初始化模型结构，还要初始化模型参数，如果直接使用Model(config)初始化方法，那就会直接导入模型结构，此时模型的参数是随机初始化的，而from-pretrained方法可以导入预训练模型的参数，即使它传入config，只会做结构上的调整，参数还是预训练模型的参数
> * 如果是PreTrainedModel，初始化需要传入一些config（不同模型构建的时候需要不同的模型参数，比如label2id代表的输出层的大小），PreTrainedModel包含forward前向传播过程，forawrd默认返回logits（logits就是最终的全连接层的输出，表示网络最后一层的输出），如果有标签，还会返回loss
> * 整个PreTrainedModel实际包括三部分：Configuration（模型构建的配置），backboned（模型架构），model_head（模型要应用的任务构建）
> * 有时候一些warning提示预训练权重没有被使用，一些权重随机初始化，正常，比如：bert模型的预训练头被丢弃，取而代之的是随机初始化的分类头
> * 构建一个model使用from_pretrained()可以导入hub上的模型（模型默认下载不是下载的原始模型，而是映射后的一些文件，通过这些文件来得到模型，所以把模型从hub上下载下来再使用），也可以导入本地checkpoint，也可以自己从零构建一个模型
> * from_pretrained方法--类方法
>> * pretrained_model_name_or_path字符串或路径,如果这个值没有，就需要同时传入config和state_dict这两个参数来构建模型,config确定结构，state_dict覆盖参数
>> * model_args 模型参数，传入到backbond的，而且这个是位置参数，只在pretrained_model_name_or_path存在时传入模型，做定义结构的作用
>> * config 模型配置，如果没有会自动导入
>> * state_dict 参数
>> * revision hub上的模型版本
>> * 出现warning：  The warning *Weights from XXX not initialized from pretrained model* 意味着XXX部分的权重是随机的，不是预训练模型中的，可以通过下游任务进行调参。The warning *Weights from XXX not used in YYY* 意味着在YYY这个模型中的XXX层的参数不被使用，会被丢弃。假设有一个模型A和预训练模型B，如果A比B大几层，那这几层就会报第一个warning，如果A比B少一些层，那这些层就会报第二个warning
>> * ignore_mismatched_sizes 默认False，是否忽略上面的两个warning
>> * output_loading_info 是否返回一个包含各种信息的字典，默认False
>> * 大模型相关的一些参数
>>> * low_cpu_mem_usage 加载模型时，尽量不要在 CPU 内存（包括峰值内存）中使用超过 1 倍的模型大小
>>> * torch_dtype 模型的参数类型，默认使用原始的参数类型，可以量化使用？？可以训练？？
>>> * device_map 设备配置， A map that specifies where each submodule should go，It doesn't need to be refined to each parameter/buffer name, once a given module name is inside, every submodule of it will be sent to the same device.也就是把模型的子模块放到不同机器上,(`str` or `Dict[str, Union[int, str, torch.device]]`,
>>> * max_memory 一个内存限制的字典，可以限制cpu的内存和gpu的内存
>>> * offload_folder ,offload_state_dict
>>> * load_in_8bit 导入模型时使用量化模型，会有性能惩罚，但是还好，注意量化后的模型不能训练
>>> * quantization_config量化配置：llm_int8_skip_modules，load_in_8bit_threshold，llm_int8_enable_fp32_cpu_offload


### (1)导入已有的预训练模型

In [3]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
# 注意可以传入模型自身的独特参数以及一些通用模型参数

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### (2)从零构建一个PreTrainedModel
> * Configuration
>> * PretrainedConfig，模型的属性，不同模型有不同的属性，比如nlp模型的attention_head,这些属性或者说参数是和模型结构有关的 ，可以修改这些属性来自定义模型架构或者改变已有模型架构（一定程度上）
>> * 你的PretrainedConfig的__init__必须接受任何`kwargs, super().__init__(**kwargs)`,那些kwargs需要传递给超类__init__
>> * 继承是为了确保您从  Transformers 库中获得所有功能，而其他两个约束来自 PretrainedConfig具有比您设置的字段更多的字段

> * Backboned
>> * 模型架构，torch.nn.Module,需要传入config，可以自定义forward和模型结构
>> * 可以把config载入model，但载入的参数是随机值而不是预训练权重，所以载入预训练要用from_pretrained
>> * 如果载入预训练模型时，同时载入config，那就相当于改了预训练模型的部分结构参数
>> * forward返回一个dict，里面必须包含logits隐层结果，如果包含loss，那么则可以在trainer中使用
>> * 输入原始hidden-state或者features的网络（包含embedding和layers），通常连接到一个接收feature的任务head来做一个预测

> * ModelHead
>> * 模型任务头，用来匹配不同的任务，Backboned的隐藏状态作为输入传递给模型头以产生最终输出,就是在原始模型基础上改了下forward
> * 其他注意
>> * 自定义配置,自定义的时候要注意继承，所以我们重新写模型配置是append模式


In [ ]:
from transformers import PretrainedConfig
from typing import List


class ResnetConfig(PretrainedConfig):
    model_type = "resnet"

    def __init__(
        self,
        block_type="bottleneck",
        layers: List[int] = [3, 4, 6, 3],
        num_classes: int = 1000,
        input_channels: int = 3,
        cardinality: int = 1,
        base_width: int = 64,
        stem_width: int = 64,
        stem_type: str = "",
        avg_down: bool = False,
        **kwargs,
    ):
        if block_type not in ["basic", "bottleneck"]:
            raise ValueError(f"`block_type` must be 'basic' or bottleneck', got {block_type}.")
        if stem_type not in ["", "deep", "deep-tiered"]:
            raise ValueError(f"`stem_type` must be '', 'deep' or 'deep-tiered', got {stem_type}.")

        self.block_type = block_type
        self.layers = layers
        self.num_classes = num_classes
        self.input_channels = input_channels
        self.cardinality = cardinality
        self.base_width = base_width
        self.stem_width = stem_width
        self.stem_type = stem_type
        self.avg_down = avg_down
        super().__init__(**kwargs)

resnet50d_config = ResnetConfig(block_type="bottleneck", stem_width=32, stem_type="deep", avg_down=True)
resnet50d_config.save_pretrained("custom-resnet")
my_config = ResnetConfig.from_pretrained("custom-resnet")


# from transformers import DistilBertConfig
#
# config = DistilBertConfig()
# my_config = DistilBertConfig(activation="relu", attention_dropout=0.4)
# my_config = DistilBertConfig.from_pretrained("distilbert-base-uncased", activation="relu", attention_dropout=0.4)
# my_config.save_pretrained(save_directory="./your_model_save_path")  # 保存 JSON 文件
# my_config = DistilBertConfig.from_pretrained("./your_model_save_path/config.json") # 使用json配置文件
# model = DistilBertModel(my_config)  # 这将创建一个具有随机值而不是预训练权重的模型

# 使用
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# model = DistilBertModel.from_pretrained("distilbert-base-uncased", config=my_config)

In [ ]:
from transformers import PreTrainedModel
from timm.models.resnet import BasicBlock, Bottleneck, ResNet
from .configuration_resnet import ResnetConfig


BLOCK_MAPPING = {"basic": BasicBlock, "bottleneck": Bottleneck}


class ResnetModel(PreTrainedModel):
    config_class = ResnetConfig

    def __init__(self, config):
        super().__init__(config)
        block_layer = BLOCK_MAPPING[config.block_type]
        self.model = ResNet(
            block_layer,
            config.layers,
            num_classes=config.num_classes,
            in_chans=config.input_channels,
            cardinality=config.cardinality,
            base_width=config.base_width,
            stem_width=config.stem_width,
            stem_type=config.stem_type,
            avg_down=config.avg_down,
        )

    def forward(self, tensor):
        return self.model.forward_features(tensor)


In [ ]:
import torch

class ResnetModelForImageClassification(PreTrainedModel):
    config_class = ResnetConfig

    def __init__(self, config):
        super().__init__(config)
        block_layer = BLOCK_MAPPING[config.block_type]
        self.model = ResNet(
            block_layer,
            config.layers,
            num_classes=config.num_classes,
            in_chans=config.input_channels,
            cardinality=config.cardinality,
            base_width=config.base_width,
            stem_width=config.stem_width,
            stem_type=config.stem_type,
            avg_down=config.avg_down,
        )

    def forward(self, tensor, labels=None):
        logits = self.model(tensor)
        if labels is not None:
            loss = torch.nn.cross_entropy(logits, labels)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}


resnet50d = ResnetModelForImageClassification(resnet50d_config)

# 导入参数
# import timm

# pretrained_model = timm.create_model("resnet50d", pretrained=True)
# resnet50d.model.load_state_dict(pretrained_model.state_dict())

## 2. 构建数据的处理器
> * 主要是tokenizer，一般使用模型对应的tokenizer，主要是把原始文本转为模型能够接收的输入
> * 这里的tokenizer不仅仅是tokenizer，还包括normalizer，pre-tokenizer,tokenizer,postprocess
> * PreTrainedTokenizer：分词器的 Python 实现。PreTrainedTokenizerFast：基于 Rust 的Tokenizers库的分词器。fast tokenizer 还提供了额外的方法，例如将标记映射到其原始单词或字符的偏移映射,支持编码和解码、添加新标记和管理特殊标记等常用方法,但并非每个模型都支持快速分词器
> * 记住，自定义分词器生成的词汇与预训练模型的分词器生成的词汇不同,如果您使用的是预训练模型，则需要使用预训练模型的词汇表，否则输入将没有意义
> * 可以导入一个已有的tokenizer，也可以自定义训练一个tokenizer（见tokenizers_learning.ipynb）

In [4]:
## 导入模型分词器
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

## 3. 处理模型需要的数据
> *要将标签映射到labels字段，这样做是因为使用Trainer进行模型训练时，会自动找到labels字段作为标签

In [5]:
train_dataset,validation_dataset = dataset.train_test_split(test_size=0.2).values()

def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True, keep_in_memory=True, batch_size=2000)
validation_dataset = validation_dataset.map(preprocess_function, batched=True, keep_in_memory=True, batch_size=2000)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 68.99ba/s]


## 4. 配置模型的data-collator
> * 默认 DataCollatorWithPadding把label列换成了labels，把label_ids列换成了labels
> * 需要pad操作，动态填充效率更高
> * 不同任务可选很多datacollator，比如DataCollatorForTokenClassification
> * 自定义data-collator的输入是features: 是一个batch大小的list，每个element是一个字典，key为模型接收的key，比如'labels', 'input_ids','token_type_ids', 'attention_mask'，输出的是模型接受的batch数据字典，每个value都是一个tensor，而且可以做padding什么的


In [6]:
# 准备data-collator, data-collector时使用padding，返回pt格式
from transformers import DataCollatorWithPadding
data_collator_padding = DataCollatorWithPadding(tokenizer=tokenizer,
                                               padding='max_length',
                                               max_length=tokenizer.model_max_length,
                                               pad_to_multiple_of=None,
                                               return_tensors="pt",
                                               )

In [ ]:
# 可以自定义data-collator
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
class Mycollator(DataCollatorWithPadding):
    def __init__(self,tokenizer,):
        super(Mycollator, self).__init__(tokenizer=tokenizer)

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        batch = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        # if "label" in batch:
        #     batch["labels"] = batch["label"]
        #     del batch["label"]
        # if "label_ids" in batch:
        #     batch["labels"] = batch["label_ids"]
        #     del batch["label_ids"]
        return batch

data_collator = Mycollator(tokenizer=tokenizer)  # 不将label删掉变成labels

## 5.配置模型训练参数
> * 数据相关的参数
>> * 'dataloader_drop_last': False,  # 是否删除最后一个不完整的batch
>> * 'dataloader_num_workers': 0,  # 多进程处理数据，载入数据进程数，0默认1个进程，只加载到主进程，告诉dataloader要创建多少个子进程进行数据加载，和cpu有关，和gpu无关
>> * 'remove_unused_columns': True,  # 自动取出多余的列
>> * 'label_names':  # label相关的列的名称，List[str]，默认["label"],如果是问答则是["start_positions", "end_positions"]
>> * 'disable_tqdm': # 是否关闭进度和metric
>> * 'ignore_data_skip':False  # 恢复训练时，是否跳过epochs和batches，让数据同时加载阶段和之前的训练一样。 如果设置为“True”，训练将开始得更快（因为跳过的步骤可能需要很长时间）但不会产生与中断训练相同的结果
>> * 'group_by_length': False,  # 默认False, 动态padding时使用，处理数据的时候把长度相同的放在一起，加速推理，训练时尽量不要用，否则短的在一起，长的在一起
>> * 'dataloader_pin_memory': True,  # 是否固定数据内存 ，默认True
>> * 其他：'past_index'

> * 训练相关的参数
>> * 'num_train_epochs': 3,  # 训练epoch
>> * 'max_steps': -1,  # 总的迭代次数，如果设置会覆盖epoch
>> * 'per_device_train_batch_size': 8,  # 每个设备（可能是GPU）的训练batch大小
>> * 'gradient_accumulation_steps': 1,  # 梯度累计step数
>> * 'weight_decay': 0,  # 权重衰减，L2正则化
>> * 'max_grad_norm': 1,  # 梯度截断，控制梯度膨胀
>> * 'no_cuda': False,  # 是否使用GPU
>> * 'seed': 42,  # 训练时的种子
>> * 'data_seed': # 数据种子，如果不设置就默认为训练的种子
>> * full_determinism # 可以确保重现分布式训练的结果，需要配合args.need，设置会True则会固定分布式的一些配置，而不是让他随机
>> * gradient_checkpointing #  如果为 True，则使用梯度检查点以较慢的反向传递为代价来节省内存。
>> * fp16 : 混合精度计算
>> * resume_from_checkpoint： 从检查点恢复训练，第一种是不使用overwrite_output_dir，会使用之前的output_dir，第二种是使用新的output_dir，此时可overwrite_output_dir，resume_from_checkpoint=True它将从最新的检查点恢复训练，resume_from_checkpoint=checkpoint_dir这将从传递的目录中的特定检查点恢复训练。
>> * sharded_ddp分布式计算，可以用
>> * fsdp 一种分片分布式计算
>> * 其他：'deepspeed',auto_find_batch_size，torchdynamo

> * 优化器和学习器相关的参数
>> * 'optim': 'adamw_hf',
>> * 'optim_args':  # 参数
>> * 'learning_rate': 5e-5,  # lr,默认使用AdamW优化器
>> * 'lr_scheduler_type': 'linear',  # 学习率优化
>> * 'adam_beta1': 0.9,
>> * 'adam_beta2': 0.999,
>> * 'adam_epsilon': 1e-8,
>> * 'warmup_ratio': 0.0,  # 预热的整个step的比例
>> * 'warmup_steps': 0,  # 预热的步数，如果设置，会覆盖比例
>> * 其他：

> * 评估相关参数
>> * 'evaluation_strategy': 'steps',  # 默认若干步评估一次，同时log输出一次
>> * 'eval_steps':  # 评估的间隔step数，如果不设置，就会和logging_steps一样
>> * 'per_device_eval_batch_size': 8,  # 每个设备（可能是GPU）的评估batch大小
>> * 'eval_accumulation_steps': , # 在把结果放到cpu上前，评估累计的次数，不设置就评估整个评估集
>> * 'eval_delay': # 评估的延迟步数，一般不用
>> * 'load_best_model_at_end': False,  # 最后载入最好的指标的那次模型，如果设置为True，save_strategy要和evaluation_strategy一样，`save_steps` 必须是 `eval_steps`的整数倍,是否在训练结束时加载在训练中找到的最佳模型，注意这个默认是根据loss来的，默认false，如果设置为True，那么所有训练完成后，会把最好结果的模型保存下来，因为训练时只会记录最好模型是哪个，而不会使用，所以最后
>> * 'metric_for_best_model':, # 评估保存的标准是什么，默认loss,训练时默认在前面加上前缀 'eval_'
>> * 'greater_is_better':True # 模型指标的选择,配合metric_for_best_model使用
>> * 'include_inputs_for_metrics': False,  # 是否把输入传入到metric计算，如果自定义的metric需要input，则需要设置这个为True
>> * 其他：

> * log相关参数
>> * 'logging_dir': './output_dir/runs',  # log日志保存在哪
>> * 'logging_strategy': 'steps',  # 日志的保存策略，默认steps
>> * 'logging_steps': 500,  # log策略是steps时的步骤数
>> * 'log_level': 'passive',  # 主进程的log的等级
>> * 'log_level_replica': 'warning',  # 重复log的等级
>> * 'log_on_each_node': True,  # 分布式训练是否log每个节点
>> * 'logging_first_step': False,  # 是否评估和log开始时的结果
>> * 'logging_nan_inf_filter': True,  # 是否过滤空值和无穷小的loss
>> * 'report_to':  上传到哪
>> * 'run_name':  名称
>> * 'skip_memory_metrics': True,  # 是否跳过内存使用分析报告，默认跳过，设置为False会出一个内存使用分析，会使得训练和评估变慢，应该只在inference测试时用，
>> * 'push_to_hub': False,
>> * 其他

> * 保存模型相关参数
>> * output_dir 必传参数，模型保存路径
>> * 'overwrite_output_dir': False,  # 是否覆写输出的路径
>> * 'save_strategy': "steps",  # 保存策略，默认steps
>> * 'save_steps': 500,  # steps时保存的步骤
>> * 'save_total_limit': 2,  # 最大存储的模型数,为了缩小最大的存储模型的个数
>> * 'save_on_each_node': False,  # 多节点训练时，是否在每个节点上存储模型，还是只在主节点上存储
>> * 其他

> * 推理相关
>> * "prediction_loss_only": False,  # 是否预测时只输出损失
>> * 'jit_mode_eval': False,  # 是否使用jit进行inference
>> * 
> * 其他：label_smoothing_factor ，do_train，do_eval，do_predict，debug,use_legacy_prediction_loop
> * 必传参数output_dir,



In [20]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # do_train=True,
    # do_eval=True,
    push_to_hub=False,
    report_to=None,
    label_names=['labels'],
    logging_steps=20,
    save_total_limit= 2
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## 6. 自定义一些其他操作


### 评估指标
> * 可调用的函数，输入的是EvalPrediction，输出的是指标字典

In [11]:
import evaluate
accuracy = evaluate.load("accuracy")
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    r = accuracy.compute(predictions=predictions, references=labels)
    return r


## 7.构建Trainer

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.select(range(200)),
    eval_dataset=validation_dataset.select(range(100)),
    tokenizer=tokenizer,
    data_collator=data_collator_padding,
    compute_metrics=compute_metrics,
)


## 8.Trainer的使用
> * train() 训练
>> * resume_from_checkpoint:从检查点继续训练，会导入model/optimizer/scheduler，默认False,可以直接输入路径
>> * trial:超参数搜索的试验数
>> * ignore_keys_for_eval：模型输出中的键列表（如果它是字典），在训练期间收集预测以进行评估时应忽略这些键。

> * evaluate() 评估数据集，返回EvalLoopOutput
>> * eval_dataset评估集
>> * ignore_keys可以忽略的指标
>> * metric_key_prefix，每个评估指标都会加'eval_'前缀
>> * 输入的是评估的预测结果，eval_pred:transformers.trainer_utils.EvalPrediction有predictions，label_ids，inputs三个属性,其中predictions和label_ids是array,而且predictions是logits

> * prediction_step 模型单步预测
>> * 输入model，inputs，prediction_loss_only,ignore_keys
>> * 返回(loss, logits, labels)
>> * evaluation_loop 用到prediction_step

> * create_optimizer_and_scheduler 创建优化器和调度器
> * get_train_dataloader
> * training_step
> * compute_loss(self, model, inputs, return_outputs=False)

> * 其他：可以自定义继承Trainer，很多方法都可以重写，模型预测的损失只会有一个值，但如果用DataParallel之类的模型，那么损失会是多个，每个是每个gpu的损失，然后gather组合起来这些，训练是每轮训练，然后每个更新step，然后每个子step
> * 默认使用ddp进行训练，取决于使用的os-gpu-env，

# 四.训练模型

## 1. 可以直接训练

In [22]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/zyl/disk/poetry_env/algorithms-ai-IgVB5U1f-py3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 39
  Number of trainable parameters = 109483778
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DIS

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.674351,0.630000
2,0.301500,0.655514,0.680000
3,0.301500,0.679990,0.660000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to my_awesome_model/checkpoint-13
Configuration saved in my_awesome_model/checkpoint-13/config.json
Model weights saved in my_awesome_model/checkpoint-13/pytorch_model.bin
tokenizer config file saved in my_awesome_model/checkpoint-13/tokenizer_config.json
Special tokens file saved in my_awesome_model/checkpoint-13/special_tokens_map.json
/home/zyl/disk/poetry_env/algorithms-ai-IgVB5U1f-py3.10/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather a

TrainOutput(global_step=39, training_loss=0.29291621232644105, metrics={'train_runtime': 23.4246, 'train_samples_per_second': 25.614, 'train_steps_per_second': 1.665, 'total_flos': 157866633216000.0, 'train_loss': 0.29291621232644105, 'epoch': 3.0})

## 1.可以自定义训练

In [ ]:
# 自定义训练
import torch
torch.cuda.empty_cache()  # 释放一些存储
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# 数据加载器
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


# 优化器，由于优化一些层的参数
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

# scheduler调度器可以指定在每个step更新学习率，但调度器不仅仅可以是学习率调度器
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 训练
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()  # 梯度计算

        optimizer.step()  # 优化器根据梯度更新参数，所以优化器指参数更新方法
        lr_scheduler.step()  # 更新超参数：比如lr，也可以其他
        optimizer.zero_grad()  # 如果不将梯度归0，则它将添加到下一步的梯度中
        progress_bar.update(1)

import evaluate
metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
metric.compute()

Method	Speed	Memory
Gradient accumulation	No	Yes
Gradient checkpointing	No	Yes
Mixed precision training	Yes	(No)
Batch size	Yes	Yes
Optimizer choice	Yes	Yes
DataLoader	Yes	No
DeepSpeed Zero	No	Yes## 2.多GPU训练
> * 设置要使用的gpu环境变量：CUDA_VISIBLE_DEVICES='0,1,2'
> * 应该使用ddp训练：`python -m torch.distributed.launch --nproc_per_node 2 run_train.py`或者torchrun:`torchrun --nproc_per_node=2 run_train.py`
> * 默认使用DP进行训练，它的速度和DDP的速度相差巨大(若干倍，取决于使用的gpu数)
> * 使用DDP的时候wandb的环境要在主进程上，使用`os.environ["WANDB_PROJECT"] = "test"`而不是wandb.init，否则可能创建多个wandb-log
> * P2P训练，要设置环境变量：export NCCL_P2P_DISABLE=1


In [ ]:
Method	Speed	Memory
Gradient accumulation	No	Yes
Gradient checkpointing	No	Yes
Mixed precision training	Yes	(No)
Batch size	Yes	Yes
Optimizer choice	Yes	Yes
DataLoader	Yes	No
DeepSpeed Zero	No	Yes

## 3.accelerate加速训练
> * Accelerator将自动检测您的分布式设置类型并初始化所有必要的训练组件。您不需要明确地将您的模型放置在设备上
> * 使用Accelerator类来加速训练，需要传入一些配置
> * 不再需要model放在gpu这种操作.不再需要数据放在gpu这种操作,使用accelerator.backward(loss)来替代loss.backward()
> * accelerate config ：create and save a configuration file
> * accelerate launch train.py ：  launch your training

In [ ]:

from accelerate import Accelerator

accelerator = Accelerator()

# # 所有相关的训练对象传递给prepare方法
# train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
#     train_dataloader, eval_dataloader, model, optimizer
# )
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         outputs = model(**batch)
#         loss = outputs.loss
#         accelerator.backward(loss)
#
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)
#
# + from accelerate import Accelerator
#   from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
#
# + accelerator = Accelerator()
#
#   model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
#   optimizer = AdamW(model.parameters(), lr=3e-5)
#
# - device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# - model.to(device)
#
# + train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
# +     train_dataloader, eval_dataloader, model, optimizer
# + )
#
#   num_epochs = 3
#   num_training_steps = num_epochs * len(train_dataloader)
#   lr_scheduler = get_scheduler(
#       "linear",
#       optimizer=optimizer,
#       num_warmup_steps=0,
#       num_training_steps=num_training_steps
#   )
#
#   progress_bar = tqdm(range(num_training_steps))
#
#   model.train()
#   for epoch in range(num_epochs):
#       for batch in train_dataloader:
# -         batch = {k: v.to(device) for k, v in batch.items()}
#           outputs = model(**batch)
#           loss = outputs.loss
# -         loss.backward()
# +         accelerator.backward(loss)
#
#           optimizer.step()
#           lr_scheduler.step()
#           optimizer.zero_grad()
#           progress_bar.update(1)


accelerate 脚本训练
accelerate config
accelerate config
accelerate launch run_summarization_no_trainer.py 

# 五.使用训练好的模型

In [23]:
checkpoint = './my_awesome_model/checkpoint-39'

## 1.单独使用
> 需要自己构建tokenizer，数据处理，和模型前向传播，不是很推荐

In [24]:
use_model = BertForSequenceClassification.from_pretrained(checkpoint)
# for epoch in range(num_epochs):
#       for batch in train_dataloader:
# -         batch = {k: v.to(device) for k, v in batch.items()}
#           outputs = model(**batch)
#           loss = outputs.loss
# -         loss.backward()
# +         accelerator.backward(loss)

loading configuration file ./my_awesome_model/checkpoint-39/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./my_awesome_model/checkpoint-39/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights

## 2.pipeline
> * 可以使用已有的pipeline，也可以根据需求自己构建pipeline
> * 管道pipeline主要用于inference，包含预处理，后处理和模型推理，相当于api,使用时，直接调用，可以输入list
> * 注意：管道的批处理并不会更好，因为如果一个批次里面出现一个长句子，那么整个批次的token大小就会是这个长token，如果数据很大，可以分块进行处理
> * 批处理只有在特定情况下才能完全发挥作用，比如gpu很多很大
> * 可以使用accelerate加速大模型的推理
> * 有preprocess，forward，postprocess可以分开来使用，所以有多个前向传播或复杂情况可以用这个
> * 这个pipeline仅支持单gpu预测,更多的是偏向在在线流处理，如果有已确定的很多数据要进行预测，还是要使用trainer-predict-多gpu预测
> * 如果模型对于单个 GPU 来说太大，可以设置device_map="auto"允许 Accelerate自动确定如何加载和存储模型权重,device和device_map不共存

In [32]:
from transformers import pipeline
classifier = pipeline('text-classification',checkpoint)
generator = pipeline(model="openai/whisper-large", device=0)


loading configuration file ./my_awesome_model/checkpoint-39/config.json
Model config BertConfig {
  "_name_or_path": "./my_awesome_model/checkpoint-39",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file ./my_awesome_model/checkpoint-39/config.json
Model config BertConfig {
  "_name_or_path": "./my_awesome_model/checkpoint-39",
  "archi

In [ ]:
# 如果处理很多数据，使用数据迭代器
def data():
    for i in range(1000):
        yield f"My example {i}"


pipe = pipeline(model="gpt2", device=0)
generated_characters = 0
for out in pipe(data()):
    generated_characters += len(out[0]["generated_text"])

# 使用大模型
# pip install accelerate
import torch
from transformers import pipeline

pipe = pipeline(model="facebook/opt-1.3b", torch_dtype=torch.bfloat16, device_map="auto")
output = pipe("This is a cool example!", do_sample=True, top_p=0.95)


In [33]:
print(classifier(dataset[0]['sentence']))
print(classifier.preprocess(dataset[0]['sentence']))
print(classifier.forward(classifier.preprocess(dataset[0]['sentence'])))
print(classifier.postprocess(classifier.forward(classifier.preprocess(dataset[0]['sentence']))))

[{'label': 'LABEL_1', 'score': 0.8845857381820679}]
{'input_ids': tensor([[  101,  1996, 11279,  8469,  1996,  9478,  3154,  1997,  1996,  5749,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
ModelOutput([('logits', tensor([[-1.2044,  0.8322]]))])
{'label': 'LABEL_1', 'score': 0.8845857381820679}


In [ ]:
# 加速accelerate
# pip install accelerate bitsandbytes
# import torch
# from transformers import pipeline

# pipe = pipeline(model="facebook/opt-1.3b", device_map="auto", model_kwargs={"load_in_8bit": True})
# output = pipe("This is a cool example!", do_sample=True, top_p=0.95

## 3. 自己构建pipeline
> * 这个

In [ ]:
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        # 这个函数为了让用户可以随时传递任何参数
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]

        postprocess_kwargs = {}
        if "top_k" in kwargs:
            postprocess_kwargs["top_k"] = kwargs["top_k"]
        return preprocess_kwargs, {}, postprocess_kwargs
    
    
    def preprocess(self, inputs, maybe_arg=2):
        model_input = Tensor(inputs["input_ids"])
        return {"model_input": model_input}

    def _forward(self, model_inputs):
        # model_inputs == {"model_input": model_input}
        outputs = self.model(**model_inputs)
        # Maybe {"logits": Tensor(...)}
        return outputs

    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"].softmax(-1)
        return best_class

my_pipeline = MyPipeline(model=model, tokenizer=tokenizer, ...)
my_pipeline = pipeline(model="xxxx", pipeline_class=MyPipeline)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification")


def data():
    while True:
        # This could come from a dataset, a database, a queue or HTTP request
        # in a server
        # Caveat: because this is iterative, you cannot use `num_workers > 1` variable
        # to use multiple threads to preprocess data. You can still have 1 thread that
        # does the preprocessing while the main runs the big inference
        yield "This is a test"


for out in pipe(data()):
    print(out)
    # {"text": "NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND"}
    # {"text": ....}
    # ....

# 六.其他
deepspeed


flexflow


benchmarkert 需要一个计算时间和内存使用的统计追踪功能


torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.device("cuda:1,3" if torch.cuda.is_available() else "cpu")  ## specify the GPU id's, GPU id's start from 0.



In [ ]:
冻结层
for param in model.bert.parameters():
    param.requires_grad = False


for name, param in model.named_parameters():
     print(name, param.requires_grad)

for name, param in model.named_parameters():
     if name.startswith("..."): # choose whatever you like here
        param.requires_grad = False

    


要冻结层就要从原始结构中进行冻结，就是找到整个模型的基准模型中的层进行冻结，可以指定层
还有，要注意如果冻结了层，但是在trian模式的话，有dropout，所以有输出不同的情况，所以要验证这个冻结层到底有没有冻结，请在eval模式进行


from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
One gets the most efficient performance when batch sizes and input/output neuron counts are divisible by a certain number,

In [ ]:
当批量大小和输入/输出神经元计数可以被某个数字(8)整除时，一个人获得最有效的性能，这个数字通常从 8 开始，但也可以更高。该数字根据所使用的特定硬件和模型的数据类型而有很大差异。

选择词汇量为 8 的倍数可显着提高性能
batch-size大小为 8 的倍数
batch-size 例如，对于 fp16，建议使用 8 的倍数，但在 A100 上为 64

如果我们想以 64 的批量大小进行训练，我们不应该使用per_device_train_batch_size=1andgradient_accumulation_steps=64而是使用per_device_train_batch_size=4和 ，gradient_accumulation_steps=16它具有相同的有效批量大小，同时可以更好地利用可用的 GPU 资源。

所以训练时 先确定要使用batch-size大小（8的倍数），若太大，则使用梯度累积

即使我们将批量大小设置为 1 并使用梯度累积，我们在处理大型模型时仍然会耗尽内存。为了在反向传播过程中计算梯度，通常会保存来自正向传播的所有激活。这会产生很大的内存开销。或者，可以忘记前向传播过程中的所有激活，并在后向传播过程中根据需要重新计算它们。然而，这会增加大量的计算开销并减慢训练速度。

梯度检查点在两种方法之间取得了折衷，并在整个计算图中保存了策略性选择的激活，因此只需为梯度重新计算一小部分激活。请参阅这篇解释梯度检查点背后思想的精彩文章。

使用梯度累积的时候使用梯度检查点，这节省了更多内存，但同时训练速度变慢了。一般的经验法则是，梯度检查点会使训练速度减慢约 20%。让我们看看另一种可以恢复速度的方法：混合精度训练


混合精度训练的想法是，并非所有变量都需要以完整（32 位）浮点精度存储。如果我们可以降低变量的精度，它们的计算就会更快。以下是影响内存使用和吞吐量的常用浮点数据类型的选择：

fp32 ( float32)
fp16 ( float16)
bf16 ( bfloat16)
tf32（CUDA 内部数据类型）

混合精度训练的想法是，并非所有变量都需要以完整（32 位）浮点精度存储。如果我们可以降低变量的精度，它们的计算就会更快。主要优势来自于以一半（16 位）精度保存激活。尽管梯度也是以半精度计算的，但它们会在优化​​步骤中转换回全精度，因此此处不节省内存。由于模型以 16 位和 32 位精度存在于 GPU 上，因此可以使用更多 GPU 内存（GPU 上原始模型的 1.5 倍），尤其是对于小批量。由于有些计算是全精度的，有些是半精度的，因此这种方法也称为混合精度训练。启用混合精度训练也只是将fp16标志设置为True:
    
    如果您可以使用 Ampere 或更新的硬件，则可以使用 bf16 进行训练和评估。虽然 bf16 的精度比 fp16 差，但它的动态范围要大得多。因此，如果过去您在训练模型时遇到溢出问题，bf16 将在大多数情况下防止这种情况发生。请记住，在 fp16 中，您可以拥有的最大数字是“65535”，超过该数字的任何数字都会溢出。bf16 数字可以大到“3.39e+38”（！），这与 fp32 大致相同——因为两者都有 8 位用于数值范围。
    
    用于训练 Transformer 模型的最常见优化器是 Adam 或 AdamW（具有权重衰减的 Adam）。Adam 通过存储先前梯度的滚动平均值来实现良好的收敛，但是，这会增加模型参数数量级的额外内存占用。对此的一种补救措施是使用替代优化器，例如 Adafactor，它对某些模型效果很好，但通常存在不稳定问题。
    
    Accelerate加速
    
    
    
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

dataloader = DataLoader(ds, batch_size=training_args.per_device_train_batch_size)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator(fp16=training_args.fp16)
model, optimizer, dataloader = accelerator.prepare(model, adam_bnb_optim, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
    loss = model(**batch).loss
    loss = loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
    
达到高训练速度的重要要求之一是能够以 GPU 可以处理的最大速度提供数据。默认情况下，一切都发生在主进程中，它可能无法足够快地从磁盘读取数据，从而造成瓶颈，导致 GPU 利用率不足。

DataLoader(pin_memory=True, ...)这确保数据被预加载到 CPU 上的固定内存中，并且通常会导致从 CPU 到 GPU 内存的传输速度更快。
DataLoader(num_workers=4, ...)- 生成多个工作人员以更快地预加载数据 - 在训练期间观察 GPU 利用率统计数据，如果距离 100% 还很远，则尝试增加工作人员数量。当然，问题可能出在其他地方，因此大量的 worker 不一定会带来更好的性能。


当我们训练模型时，我们希望同时优化两个方面：

数据吞吐量/训练时间
模型性能
我们已经看到每种方法都会改变内存使用和吞吐量。一般来说，我们希望最大化吞吐量（样本/秒）以最小化训练成本。这通常是通过尽可能多地利用 GPU 并将 GPU 内存填充到极限来实现的。例如，如前所述，只有当我们想要使用超过 GPU 内存大小时的批量大小时，我们才使用梯度累积。如果所需的批量大小适合内存，则没有理由应用只会减慢训练速度的梯度累积。

第二个目标是模型性能。仅仅因为我们可以并不意味着我们应该使用大批量。作为超参数调整的一部分，您应该确定哪个批量大小会产生最佳结果，然后相应地优化吞吐量。

DataParallel (DP) - 相同的设置被复制多次，并且每次都被馈送数据的一部分。处理是并行完成的，所有设置在每个训练步骤结束时同步。
TensorParallel (TP) - 每个张量被分成多个块，因此不是让整个张量驻留在单个 gpu 上，而是张量的每个分片都驻留在其指定的 gpu 上。在处理过程中，每个分片在不同的 GPU 上分别并行处理，结果在步骤结束时同步。这就是所谓的水平并行性，因为拆分发生在水平面上。
PipelineParallel (PP) - 模型在多个 GPU 上垂直（层级）拆分，因此只有一个或多个模型层位于单个 gpu 上。每个 gpu 并行处理管道的不同阶段，并处理一小部分批处理。
零冗余优化器 (ZeRO) - 也执行张量的分片，有点类似于 TP，除了整个张量会及时重建以进行前向或反向计算，因此不需要修改模型。它还支持各种卸载技术以补偿有限的 GPU 内存。
Sharded DDP - 是 ZeRO 的各种其他实现所使用的基础 ZeRO 概念的另一个名称。

In [ ]:

推理
使用 device_map = 'auto'，to_bettertransformer

model = model.to_bettertransformer()

model = model.reverse_bettertransformer()
model.save_pretrained("saved_model")

from transformers import AutoModelForCausalLM

model_name = "bigscience/bloom-2b5"
model_8bit = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)


max_memory_mapping = {0: "600MB", 1: "1GB"}
model_name = "bigscience/bloom-3b"
model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", load_in_4bit=True, max_memory=max_memory_mapping
    
    
)


from transformers import AutoModelForCausalLM

model_name = "bigscience/bloom-2b5"
model_8bit = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloom-2b5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_8bit = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

prompt = "Hello, my llama is cute"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
generated_ids = model.generate(**inputs)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
从 4.18.0 版开始，最终占用超过 10GB 空间的模型检查点会自动分成更小的部分。就拥有一个检查点而言model.save_pretrained(save_dir)，当你这样做时，你最终会得到几个部分检查点（每个检查点的大小 < 10GB）和一个将参数名称映射到它们存储的文件的索引。

您可以在使用参数进行分片之前控制最大尺寸max_shard_size，因此为了举例说明，我们将使用分片尺寸较小的正常尺寸模型：让我们采用传统的 BERT 模型。

from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")

import os
import tempfile

with tempfile.TemporaryDirectory() as tmp_dir:
    model.save_pretrained(tmp_dir)
    print(sorted(os.listdir(tmp_dir)))
    
with tempfile.TemporaryDirectory() as tmp_dir:
    model.save_pretrained(tmp_dir, max_shard_size="200MB")
    print(sorted(os.listdir(tmp_dir)))
    
from transformers.modeling_utils import load_sharded_checkpoint

with tempfile.TemporaryDirectory() as tmp_dir:
    model.save_pretrained(tmp_dir, max_shard_size="200MB")
    load_sharded_checkpoint(model, tmp_dir)
    
with tempfile.TemporaryDirectory() as tmp_dir:
model.save_pretrained(tmp_dir, max_shard_size="200MB")
new_model = AutoModel.from_pretrained(tmp_dir)

In [ ]:
class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            print(logs)
            
 https://github.com/huggingface/transformers/blob/v4.30.0/src/transformers/trainer_callback.py#L227
https://github.com/huggingface/transformers/blob/v4.30.0/src/transformers/trainer_callback.py#L227
自定义回调函数

class MyCallback(TrainerCallback):
    "A callback that prints a message at the beginning of training"

    def on_train_begin(self, args, state, control, **kwargs):
        print("Starting training")


trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[MyCallback],  # We can either pass the callback class this way or an instance of it (MyCallback())
)

trainer = Trainer(...)
trainer.add_callback(MyCallback)
# Alternatively, we can pass an instance of the callback class
trainer.add_callback(MyCallback())

In [ ]:
bitsandbytes 量化
# pip install transformers accelerate bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "bigscience/bloom-1b7"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True)

#####
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "bigscience/bloom-1b7"

quantization_config = BitsAndBytesConfig(
    llm_int8_threshold=10,
)

model_8bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device_map,
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)